<a href="https://colab.research.google.com/github/yoonhero/Brainstormers/blob/master/paperwithcode/ResNet_Deep_Residual_Learning_for_Image_Recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as T

import os
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
device = torch.device("cuda")

In [18]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential() # if x identity

        if stride != 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)

        return out

In [26]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1) # 처음 레이어에서만 w h 줄어들도록 
        layers = []

        for stride in strides: 
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
        

In [27]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [28]:
transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

Files already downloaded and verified


In [35]:
device = torch.device("cuda")

net = ResNet18().to(device)

learning_rate = 0.1

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()

        pred = net(x)
        loss = criterion(pred, y)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = pred.max(1)

        total += x.size(0)
        correct += predicted.eq(y).sum().item()

    acc = correct / total
    train_loss = train_loss / len(train_loader)
    print(f"Epoch [{epoch}] | Train Loss: {train_loss} | Accuracy : {acc}")



In [36]:
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [37]:
for epoch in range(20):
    adjust_learning_rate(optimizer, epoch)   
    train(epoch)

Epoch [0] | Train Loss: 1.797032809318484 | Accuracy : 0.3444
Epoch [1] | Train Loss: 1.3124886889896734 | Accuracy : 0.52248
Epoch [2] | Train Loss: 1.041722156812468 | Accuracy : 0.6287
Epoch [3] | Train Loss: 0.8740182008279864 | Accuracy : 0.69074
Epoch [4] | Train Loss: 0.7367094144644335 | Accuracy : 0.73914
Epoch [5] | Train Loss: 0.6396203605110383 | Accuracy : 0.77822
Epoch [6] | Train Loss: 0.5618307292461395 | Accuracy : 0.8064
Epoch [7] | Train Loss: 0.5019640300584876 | Accuracy : 0.82722
Epoch [8] | Train Loss: 0.46653542806730247 | Accuracy : 0.83842
Epoch [9] | Train Loss: 0.43487626878196933 | Accuracy : 0.84908
Epoch [10] | Train Loss: 0.40719012020494016 | Accuracy : 0.85854
Epoch [11] | Train Loss: 0.3782024359535378 | Accuracy : 0.86972
Epoch [12] | Train Loss: 0.3594250238841147 | Accuracy : 0.877
Epoch [13] | Train Loss: 0.3453153218226055 | Accuracy : 0.8813
Epoch [14] | Train Loss: 0.3278406640833906 | Accuracy : 0.88656
Epoch [15] | Train Loss: 0.3179916718121